In [ ]:
from unsloth import FastModel
import torch

# More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-07 02:53:39 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-07 02:53:39 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A10G. Num GPUs = 1. Max memory: 22.184 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [2]:
model

Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
            

Add LoRA adapters so that we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


## Data preparation
Use Gemma-3 format for conversation style finetunes.
```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use Unsloth's get_chat_template function to get the correct chat template. Unsloth supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3 and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [5]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [6]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

Inspect the dataset

In [7]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

In [8]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }

dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

Notice that the 'text' component has been reformatted to be like Gemma-3's requirement.

In [20]:
from IPython.display import display, Markdown

display(Markdown(dataset[100]['text']))

<bos><start_of_turn>user
What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>
<start_of_turn>model
In programming, the modulus operator is represented by the '%' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:

```python
# Calculate the modulus
Modulus = a % b

print("Modulus of the given numbers is: ", Modulus)
```

In this code snippet, the variables 'a' and 'b' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator '%', we calculate the remainder when 'a' is divided by 'b'. The result is then stored in the variable 'Modulus'. Finally, the modulus value is printed using the 'print' statement.

For example, if 'a' is 10 and 'b' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:

```
Modulus of the given numbers is: 2
```

This means that the modulus of 10 and 4 is 2.<end_of_turn>


## Train the model

We use HuggingFace's TRL (transformer reinforcement learning) framework's `SFTTrainer` (Supervised Finetuning Trainer)

> Set `num_train_epochs=1` for a full run and turn off `max_steps=None`

In [15]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

We also use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [16]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

Verify the masking instruction part - printing the 100th row

In [28]:
trainer.train_dataset[100].keys()

dict_keys(['conversations', 'source', 'score', 'text', 'input_ids', 'attention_mask', 'labels'])

In [29]:
trainer.train_dataset[100]['labels'][:10]

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]

In [ ]:
display(Markdown(tokenizer.decode(trainer.train_dataset[100]["input_ids"])))

<bos><bos><start_of_turn>user
What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>
<start_of_turn>model
In programming, the modulus operator is represented by the '%' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:

```python
# Calculate the modulus
Modulus = a % b

print("Modulus of the given numbers is: ", Modulus)
```

In this code snippet, the variables 'a' and 'b' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator '%', we calculate the remainder when 'a' is divided by 'b'. The result is then stored in the variable 'Modulus'. Finally, the modulus value is printed using the 'print' statement.

For example, if 'a' is 10 and 'b' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:

```
Modulus of the given numbers is: 2
```

This means that the modulus of 10 and 4 is 2.<end_of_turn>


Printing the masked out example - should only see the answer present

In [21]:
display(Markdown(tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")))

                                In programming, the modulus operator is represented by the '%' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:

```python
# Calculate the modulus
Modulus = a % b

print("Modulus of the given numbers is: ", Modulus)
```

In this code snippet, the variables 'a' and 'b' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator '%', we calculate the remainder when 'a' is divided by 'b'. The result is then stored in the variable 'Modulus'. Finally, the modulus value is printed using the 'print' statement.

For example, if 'a' is 10 and 'b' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:

```
Modulus of the given numbers is: 2
```

This means that the modulus of 10 and 4 is 2.<end_of_turn>


Show current memory statistics

In [30]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10G. Max memory = 22.184 GB.
4.809 GB of memory reserved.


In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.186100
2,1.651400
3,1.702900
4,1.383800
5,1.175000
6,1.533600
7,0.813300
8,1.182100
9,0.940900
10,0.824100


Final memory and time stats

In [32]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

180.7446 seconds used for training.
3.01 minutes used for training.
Peak reserved memory = 6.17 GB.
Peak reserved memory for training = 1.361 GB.
Peak reserved memory % of max memory = 27.813 %.
Peak reserved memory for training % of max memory = 6.135 %.


## Inference
According to the Gemma-3 team, the recommended settings for inference are temperature = 1.0, top_p = 0.95, top_k = 64

In [34]:
from transformers import TextStreamer

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

## This 'model' is already with the finetuned LoRA adapters!
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sky is blue due to a phenomenon called Rayleigh scattering. This is when sunlight interacts with the atmosphere and is scattered into different colors. Blue light has a shorter wavelength and is scattered more, which is why we see blue. 

Other factors such as the thickness of the atmosphere and the angle of the sun can affect


## Saving

In [36]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [37]:
model.push_to_hub(
    "tituslhy/gemma-3-finetuned", token=os.environ["HUGGINGFACE_ACCESS_TOKEN"]
)

README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

Saved model to https://huggingface.co/tituslhy/gemma-3-finetuned


In [38]:
tokenizer.push_to_hub(
    "tituslhy/gemma-3-finetuned", token=os.environ["HUGGINGFACE_ACCESS_TOKEN"]
)

  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

## Load just the LoRA adapters

In [42]:
model, tokenizer = FastModel.from_pretrained(
    model_name="tituslhy/gemma-3-finetuned",
    max_seq_length=4096,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A10G. Num GPUs = 1. Max memory: 22.184 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [43]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?"
            }
        ]
    }
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, #must add for generation
)

In [45]:
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The modulus operator in programming is used to find the remainder of a division of two numbers. It returns the remainder after the division is performed without any remainder. It is often used to calculate the modulus of two given numbers.
The modulus operator is represented by the following notation:

```cpp
remainder = a % b
```
Here, a represents the dividend and b represents the divisor, and the remainder is the value that the modulo operator returns.

Example:
Let's calculate the modulus of two numbers (10 and 3).

```cpp
int dividend = 10;
int divisor = 3;
int remainder = dividend % divisor;

cout << remainder;
// Output: 1
```
In this case, the remainder is 1. The modulo operator calculates the remainder when the dividend is divided by the divisor. In this example, the division of 10 by 3 is equal to 3 with a remainder of 1.

You can calculate the remainder by performing the division operation and then subtracting the result from the dividend:

```cpp
remainder = dividend - (div